# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [21]:
tf.__version__

'1.14.0'

In [22]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Importar o dataset

In [23]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

('X >>\n', array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object))
('y >>\n', array([1, 0, 1, ..., 1, 1, 0]))


### Transformando os dados categóricos


In [24]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [27]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [28]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [29]:
ann.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

### Adding the second hidden layer

In [30]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

In [31]:
ann.add(tf.keras.layers.Dense(units=6, activation='softplus'))

### Adding the output layer

In [32]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Part 3 - Training the ANN

### Compiling the ANN

In [33]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [34]:
ann.fit(X_train, y_train, batch_size = 40, epochs = 150)

Epoch 1/150
8000/8000 [==============================] - 1s 64us/sample - loss: 0.5413 - acc: 0.7291
Epoch 2/150
8000/8000 [==============================] - 0s 28us/sample - loss: 0.4641 - acc: 0.7960
Epoch 3/150
8000/8000 [==============================] - 0s 34us/sample - loss: 0.4531 - acc: 0.7985
Epoch 4/150
8000/8000 [==============================] - 0s 28us/sample - loss: 0.4460 - acc: 0.8043
Epoch 5/150
8000/8000 [==============================] - 0s 27us/sample - loss: 0.4352 - acc: 0.8071
Epoch 6/150
8000/8000 [==============================] - 0s 34us/sample - loss: 0.4322 - acc: 0.8105
Epoch 7/150
8000/8000 [==============================] - 0s 35us/sample - loss: 0.4343 - acc: 0.8139
Epoch 8/150
8000/8000 [==============================] - 0s 35us/sample - loss: 0.4282 - acc: 0.8126
Epoch 9/150
8000/8000 [==============================] - 0s 37us/sample - loss: 0.4250 - acc: 0.8155
Epoch 10/150
8000/8000 [==============================] - 0s 31us/sample - loss: 0.4311 - a

8000/8000 [==============================] - 0s 26us/sample - loss: 0.3361 - acc: 0.8624
Epoch 82/150
8000/8000 [==============================] - 0s 29us/sample - loss: 0.3371 - acc: 0.8627
Epoch 83/150
8000/8000 [==============================] - 0s 30us/sample - loss: 0.3340 - acc: 0.8639
Epoch 84/150
8000/8000 [==============================] - 0s 28us/sample - loss: 0.3344 - acc: 0.8633
Epoch 85/150
8000/8000 [==============================] - 0s 26us/sample - loss: 0.3355 - acc: 0.8637
Epoch 86/150
8000/8000 [==============================] - 0s 30us/sample - loss: 0.3346 - acc: 0.8631
Epoch 87/150
8000/8000 [==============================] - 0s 30us/sample - loss: 0.3370 - acc: 0.8608
Epoch 88/150
8000/8000 [==============================] - 0s 27us/sample - loss: 0.3354 - acc: 0.8633
Epoch 89/150
8000/8000 [==============================] - 0s 26us/sample - loss: 0.3365 - acc: 0.8622
Epoch 90/150
8000/8000 [==============================] - 0s 31us/sample - loss: 0.3376 - acc: 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [35]:
y_pred = ann.predict(X_test)
y_pred

array([[0.21616367],
       [0.16609228],
       [0.2571455 ],
       ...,
       [0.14394057],
       [0.1450317 ],
       [0.20854092]], dtype=float32)

In [36]:
y_pred = (y_pred > 0.9)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [37]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1593    2]
 [ 358   47]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)